# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [20]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [21]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

There might be a problem with your API key? Please visit the troubleshooting notebook!


In [22]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [23]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [24]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [25]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [26]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [27]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'nebula project',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [28]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    print(result)
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [29]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
{
  "links": [
    {"type": "home page", "url": "https://edwarddonner.com/"},
    {"type": "about page", "url": "https://edwarddonner.com/about-me-and-about-nebula/"},
    {"type": "project page", "url": "https://edwarddonner.com/connect-four/"},
    {"type": "product page", "url": "https://edwarddonner.com/outsmart/"},
    {"type": "blog page", "url": "https://edwarddonner.com/posts/"},
    {"type": "company page", "url": "https://nebula.io/?utm_source=ed&utm_medium=referral"},
    {"type": "patent", "url": "https://patents.google.com/patent/US20210049536A1/"},
    {"type": "linkedin", "url": "https://www.linkedin.com/in/eddonner/"},
    {"type": "twitter", "url": "https://twitter.com/edwarddonner"},
    {"type": "facebook", "url": "https://www.facebook.com/edward.donner.52"}
  ]
}
Found 10 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'product page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [30]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
{
  "links": [
    {"type": "about page", "url": "https://huggingface.co/brand"},
    {"type": "careers page", "url": "https://apply.workable.com/huggingface/"},
    {"type": "homepage", "url": "https://huggingface.co/"},
    {"type": "social media", "url": "https://twitter.com/huggingface"},
    {"type": "social media", "url": "https://www.linkedin.com/company/huggingface/"},
    {"type": "code repository", "url": "https://github.com/huggingface"},
    {"type": "community forum", "url": "https://discuss.huggingface.co"},
    {"type": "blog", "url": "https://huggingface.co/blog"},
    {"type": "community join", "url": "https://huggingface.co/join/discord"}
  ]
}
Found 9 relevant links


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'homepage', 'url': 'https://huggingface.co/'},
  {'type': 'social media', 'url': 'https://twitter.com/huggingface'},
  {'type': 'social media',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'code repository', 'url': 'https://github.com/huggingface'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community join', 'url': 'https://huggingface.co/join/discord'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [31]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [32]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
{
  "links": [
    {"type": "homepage", "url": "https://huggingface.co/"},
    {"type": "brand/about page", "url": "https://huggingface.co/brand"},
    {"type": "enterprise page", "url": "https://huggingface.co/enterprise"},
    {"type": "careers page", "url": "https://apply.workable.com/huggingface/"},
    {"type": "pricing page", "url": "https://huggingface.co/pricing"},
    {"type": "blog", "url": "https://huggingface.co/blog"},
    {"type": "documentation", "url": "https://huggingface.co/docs"},
    {"type": "learn resources", "url": "https://huggingface.co/learn"},
    {"type": "community forum", "url": "https://discuss.huggingface.co"},
    {"type": "status page", "url": "https://status.huggingface.co/"},
    {"type": "GitHub", "url": "https://github.com/huggingface"},
    {"type": "LinkedIn", "url": "https://www.linkedin.com/company/huggingface/"},
    {"type": "Twitter", "url": "https://twitter.com/huggin

In [36]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [37]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [38]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
{
  "links": [
    {"type": "home page", "url": "https://huggingface.co/"},
    {"type": "brand page", "url": "https://huggingface.co/brand"},
    {"type": "company page", "url": "https://huggingface.co/huggingface"},
    {"type": "enterprise page", "url": "https://huggingface.co/enterprise"},
    {"type": "careers page", "url": "https://apply.workable.com/huggingface/"},
    {"type": "GitHub page", "url": "https://github.com/huggingface"},
    {"type": "Twitter profile", "url": "https://twitter.com/huggingface"},
    {"type": "LinkedIn page", "url": "https://www.linkedin.com/company/huggingface/"},
    {"type": "Community forum", "url": "https://discuss.huggingface.co"},
    {"type": "Product: Endpoints", "url": "https://endpoints.huggingface.co"},
    {"type": "Blog", "url": "https://huggingface.co/blog"},
    {"type": "Learn center", "url": "https://huggingface.co/learn"},
    {"type": "Discord community", "ur

'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2-Thinking\nUpdated\nabout 1 hour ago\n•\n12.5k\n•\n595\nMiniMaxAI/MiniMax-M2\nUpdated\nabout 24 hours ago\n•\n854k\n•\n1.18k\ndx8152/Qwen-Edit-2509-Multiple-angles\nUpdated\n1 day ago\n•\n16.7k\n•\n357\nmaya-research/maya1\nUpdated\n1 day ago\n•\n4.58k\n•\n283\ndeepseek-ai/DeepSeek-OCR\nUpdated\n4 days ago\n•\n2.89M\n•\n2.54k\nBrowse 1M+ models\nSpaces\nRunning\non\nCPU Up

In [39]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [40]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
{
  "links": [
    {"type": "home page", "url": "https://huggingface.co/"},
    {"type": "brand page", "url": "https://huggingface.co/brand"},
    {"type": "company page", "url": "https://huggingface.co/huggingface"},
    {"type": "enterprise page", "url": "https://huggingface.co/enterprise"},
    {"type": "pricing page", "url": "https://huggingface.co/pricing"},
    {"type": "careers page", "url": "https://apply.workable.com/huggingface/"},
    {"type": "community forum", "url": "https://discuss.huggingface.co"},
    {"type": "github page", "url": "https://github.com/huggingface"},
    {"type": "twitter", "url": "https://twitter.com/huggingface"},
    {"type": "linkedin", "url": "https://www.linkedin.com/company/huggingface/"}
  ]
}
Found 10 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the premier AI community and collaboration platform shaping the future of machine learning (ML). It serves as a central hub where ML practitioners, researchers, and developers come together to create, share, and explore open-source models, datasets, and applications across modalities like text, images, video, audio, and even 3D.

---

## What We Offer

- **Models**: Access and contribute to over **1 million machine learning models**, ranging from NLP to computer vision and beyond.
- **Datasets**: Discover and use from a repository of more than **250,000 datasets**, enabling robust training and benchmarking.
- **Spaces**: Host and deploy ML demos and applications with ease using Hugging Face’s Spaces platform. Browse over **400,000 applications** created by the community.
- **Open Source Stack**: Accelerate your projects with cutting-edge tools and libraries in a fully open-source ecosystem.
- **Enterprise Solutions**: Scalable paid Compute and tailored enterprise offerings to empower teams and organizations with advanced ML capabilities.

---

## Our Community & Culture

Hugging Face champions an **open and ethical AI future** by fostering a vibrant, fast-growing global community of machine learning engineers, scientists, hobbyists, and end-users. The platform encourages:

- **Collaboration**: Seamless sharing and joint development of state-of-the-art ML assets.
- **Learning**: Building your professional portfolio by sharing your work with the wider AI community.
- **Accessibility**: Democratizing AI by providing tools and infrastructure to move faster in ML development.
- **Ethics**: Commitment to open, ethical AI practices through transparent, community-driven innovation.

---

## Customers & Users

Hugging Face supports a diverse ecosystem, including:

- AI researchers and academic institutions exploring novel models and datasets.
- Machine learning engineers and data scientists creating production-ready applications.
- Enterprises seeking cutting-edge solutions and dedicated support to deploy AI at scale.
- Open-source contributors advancing the state of the art collaboratively.
- Enthusiasts building portfolios and engaging with thousands of community projects.

---

## Careers

Join Hugging Face and be part of the future of AI!

- Work alongside top-tier ML engineers, researchers, and community builders.
- Contribute to open source projects with global impact.
- Help build infrastructure and tools that accelerate AI development worldwide.
- Thrive in a mission-driven culture focused on innovation, openness, and ethics.

_Visit Hugging Face’s career page to explore current openings and apply._

---

## Get Started

- Explore AI apps or browse the vast catalog of models and datasets at [huggingface.co](https://huggingface.co)
- Sign up for free to start hosting your ML projects, building your profile, and joining the conversation.
- Discover enterprise solutions tailored to your business needs.

---

## Brand & Visual Identity

- Signature colors: Bright Yellow (#FFD21E), Orange (#FF9D00), and Gray (#6B7280)
- Friendly, approachable branding reflecting the collaborative spirit of the community.
- Available brand assets include logos in multiple formats for community use.

---

**Hugging Face** — Building the future of AI, together.  
Be part of the AI revolution.

---

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [41]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [42]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
{
  "links": [
    {"type": "company page", "url": "https://huggingface.co/"},
    {"type": "brand/about page", "url": "https://huggingface.co/brand"},
    {"type": "careers page", "url": "https://apply.workable.com/huggingface/"},
    {"type": "blog", "url": "https://huggingface.co/blog"},
    {"type": "github page", "url": "https://github.com/huggingface"},
    {"type": "linkedin page", "url": "https://www.linkedin.com/company/huggingface/"},
    {"type": "twitter page", "url": "https://twitter.com/huggingface"},
    {"type": "community forum", "url": "https://discuss.huggingface.co"}
  ]
}
Found 8 relevant links


# Hugging Face Company Brochure

---

## About Hugging Face

Hugging Face is the AI community building the future of machine learning. It is a leading collaboration platform where the machine learning community—engineers, scientists, researchers, and end users—come together to create, share, explore, and innovate on machine learning models, datasets, and applications.

The Hugging Face Hub acts as a central repository providing open-source resources that empower the next generation of ML professionals to learn, experiment, and build ethical AI solutions together.

---

## What We Offer

- **Hugging Face Hub:**  
  A collaborative space hosting over 1 million ML models and 250,000+ datasets spanning modalities like text, image, video, audio, and even 3D.

- **Spaces:**  
  A platform to build and deploy ML-powered applications, with thousands of live spaces showcasing innovative AI apps.

- **Open Source Stack:**  
  Tools and frameworks to help accelerate ML workflows, enabling developers and teams to move faster and build more sophisticated models.

- **Enterprise Solutions:**  
  Scalable compute infrastructure and dedicated enterprise offerings to support teams building advanced AI applications.

---

## Community & Collaboration

Hugging Face cultivates a vibrant and fast-growing global community of AI enthusiasts. The platform enables contributors to:

- Share their machine learning work openly.
- Collaborate on cutting-edge AI projects.
- Build professional ML portfolios visible to the world.
- Access and contribute to trending models and datasets updated daily.

This strong culture of openness and shared innovation drives the co-creation of the future of AI.

---

## Company Culture

- **Open & Ethical AI:** Hugging Face is committed to promoting transparency, inclusiveness, and ethics in AI development.
- **Learning & Growth:** The community platform empowers every user to learn from and contribute to the collective knowledge pool.
- **Innovation-Driven:** Constantly pushing the boundaries of what is possible with AI by supporting diverse modalities and use cases.
- **Collaborative Spirit:** Encourages a collaborative environment where imagination and expertise thrive.

---

## Customers & Use Cases

Hugging Face serves:

- **Machine Learning Engineers & Researchers:** Offering resources to accelerate development cycles and experiment efficiently.
- **Enterprises & Startups:** Providing compute and platform solutions to integrate AI into products and services at scale.
- **Data Scientists & Developers:** Empowering users to build AI-powered applications across domains such as autonomous vehicles, NLP, computer vision, and more.

Users benefit from access to a rich ecosystem of models like Qwen Image Edit, DeepSeek OCR, and datasets from notable contributors including NVIDIA.

---

## Careers at Hugging Face

Joining Hugging Face means becoming part of a vibrant AI community dedicated to open source and cutting-edge ML research. The company fosters:

- Opportunities to work on state-of-the-art AI technologies.
- A supportive environment encouraging innovation and ethical AI.
- Collaboration with some of the brightest minds in the AI community.

Prospective candidates passionate about AI, ethics, and community-driven innovation are encouraged to explore career opportunities with Hugging Face.

---

## Get Involved

- **Explore AI Apps:** Engage with thousands of applications built on the platform.
- **Browse Models and Datasets:** Access and contribute to hundreds of thousands of open-source machine learning assets.
- **Join the Community:** Sign up to build your portfolio, collaborate, and shape the future of AI.

Visit us at [huggingface.co](https://huggingface.co) and become part of the AI revolution!

---

**Hugging Face — The AI community building the future.**

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>